# Symmetry function testing
In an effort to reduce the number of allocations and memory overhead, a rewrite of the essential program with benchmarks is here. 

In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
@everywhere using InvertedIndices
@everywhere using MachineLearningPotential

First we initialise the positions, symmetry functions, distance squared matrix and cutoff functions

In [2]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)
f_mat = cutoff_function.(sqrt.(dis2mat),Ref(total_symm_vec[1].r_cut))

55×55 Matrix{Float64}:
 1.0        0.649134    0.649134    …  0.21115     0.21115     0.0959383
 0.649134   1.0         0.617699       0.0         0.0         0.0
 0.649134   0.617699    1.0            0.00208061  0.0912928   0.0
 0.649134   0.617699    0.617699       0.0         0.00208061  0.0
 0.649134   0.617699    0.223352       0.00208061  0.0         0.0
 0.649134   0.617699    0.223352    …  0.0912928   0.00208061  0.0
 0.649134   0.617699    0.617699       0.0912928   0.0912928   0.0
 0.649134   0.223352    0.617699       0.0912928   0.355548    0.185434
 0.649134   0.223352    0.223352       0.0912928   0.0912928   0.185434
 0.649134   0.223352    0.0889637      0.355548    0.0912928   0.185434
 ⋮                                  ⋱                          
 0.0959383  0.0         0.0            0.623382    0.164805    0.0613471
 0.21115    0.00208061  0.0912928      0.608792    0.608792    0.164805
 0.0959383  0.0         0.185434       0.164805    0.623382    0.0613471
 0.2

We include the true symmetry values as generated by RuNNer to check consistency, as well as an arbitrary perturbation to test the "difference matrix" version of the symmetry function code

In [3]:
perturbation = SVector(-0.6 , 0.3, 0.2 )
newpos = positions[3] .+ perturbation
index = 3

new_dis_vec = [distance2(newpos,b) for b in positions]
new_dis_vec[3] = 0.

new_f_vec = cutoff_function.(sqrt.(new_dis_vec),Ref(total_symm_vec[1].r_cut))
testpos = copy(positions)
testpos[3] = newpos
test_dist_mat,test_f_mat = copy(dis2mat),copy(f_mat)
test_dist_mat[3,:],test_f_mat[3,:] = new_dis_vec,new_f_vec
test_dist_mat[:,3],test_f_mat[:,3] = new_dis_vec,new_f_vec

([18.394380981758086, 18.774588593575903, 0.0, 17.256423589706046, 51.82642834778728, 56.42544269955387, 24.697785120746254, 22.241342541268853, 53.96900012439397, 78.17678524073806  …  142.43105031042984, 144.3017422418635, 90.39040168739315, 75.81835053377075, 84.18930839905248, 116.35219804625032, 140.89205948670923, 123.89563830625079, 88.85141088062991, 141.29424803201155], [0.686580814637355, 0.6808988520376276, 1.0, 0.7037709087072497, 0.29431511657444837, 0.25565770092848616, 0.5962795692198719, 0.6304923344540447, 0.2758906654833895, 0.11490912789423063  …  0.0, 0.0, 0.06295474344239504, 0.12704886546595062, 0.08710767926238971, 0.0058230308892733285, 0.0, 0.0008234780426423782, 0.06853545298337882, 0.0])

In [4]:
file = open("$(pwd())/symfunctions.out","r+")
truevalues = readdlm(file)
close(file)
truevals = transpose(truevalues[2:end,2:end])

88×55 transpose(::Matrix{Any}) with eltype Any:
 14.6407       11.9666       11.9666       …  7.45537      5.69954
  0.0           0.0           0.0             0.0          0.0
  7.16978       6.36224       6.36224         3.92888      2.94156
  0.0           0.0           0.0             0.0          0.0
  4.52421       4.14311       4.14311         2.54596      1.91555
  0.0           0.0           0.0          …  0.0          0.0
  0.972533      0.857168      0.857168        0.508399     0.410771
  0.0           0.0           0.0             0.0          0.0
  0.00179489    0.00117603    0.00117603      0.000528536  0.000557398
  0.0           0.0           0.0             0.0          0.0
  ⋮                                        ⋱               
  0.00742548    0.00500205    0.00500205      0.00192165   0.00165975
  0.0           0.0           0.0          …  0.0          0.0
  0.0           0.0           0.0             0.0          0.0
  0.000947854   0.000598351   0.000598351

Next, testing the speed and accuracy of the existing code.

In [5]:
gmat1 = total_symm_calc(positions,dis2mat,f_mat,total_symm_vec)

gmat2 = total_symm_calc(testpos,test_dist_mat,test_f_mat,total_symm_vec)

truevals ≈ gmat1



true

In [6]:
# testgmat1 = total_symm_calc(positions,newpos,dis2mat,new_dis_vec,f_mat,new_f_vec,gmat1,3,total_symm_vec)

# testgmat1 ≈ gmat2

This indicates that the existing "update g_matrix" code is wrong

benchmark for generating the full and partial matrix

In [7]:
@benchmark total_symm_calc($positions,$dis2mat,$f_mat,$total_symm_vec)

BenchmarkTools.Trial: 116 samples with 1 evaluation.
 Range (min … max):  40.221 ms … 50.293 ms  ┊ GC (min … max):  9.82% … 8.33%
 Time  (median):     41.544 ms              ┊ GC (median):     9.97%
 Time  (mean ± σ):   43.109 ms ±  2.823 ms  ┊ GC (mean ± σ):  11.98% ± 3.35%

     ▅▃█▅▄▃                                                    
  ▅▇▇██████▆▃▇▅▆▁▁▅▁▁▁▅▅▁▁▃▃▁▁▅▃▁▃▃▁▁▃▃▃▅▁▃▃▅▃▃▃▃▆▃▃▅▅▅▃▃▁▁▁▅ ▃
  40.2 ms         Histogram: frequency by time        49.4 ms <

 Memory estimate: 104.19 MiB, allocs estimate: 1364455.

Updating the g_matrix code below

In [8]:
# these calculate the two main components of the angular symmetry function, the exp part is also written so it can be used for both types of symmetry function

exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk) = exp(-η*(r2_ij+r2_ik+r2_jk))* f_ij * f_ik * f_jk

exponential_part(η,rsum,f_prod) = exp(-η*(rsum))*f_prod

theta_part(θ,λ,ζ) = (1+λ*θ)^ζ


theta_part (generic function with 1 method)

In [9]:

#funtions desinged to update radial and angular symmetry values

function adjust_angular_symm_val!(g_value,θ_val,exp_part,tpz)
    g_value += exp_part*θ_val*tpz
    return g_value
end

function adjust_symm_val!(g_value,r_sum,f_prod,η)
    #adjusts radial type 2 symmetry function
    g_value += exponential_part(η,r_sum,f_prod)
    return g_value
end


function adjust_angular_symm_val!(g_value,θ_new,θ_old,exp_new,exp_old,tpz)

    g_value += exp_new*θ_new*tpz
    g_value -= exp_old*θ_old*tpz

    return g_value
end

function adjust_angular_symm_val!(g_value,exp_old,exp_new,θ_old,θ_new,λ,ζ,tpz)
    θ_val_old,θ_val_new = theta_part(θ_old,λ,ζ),theta_part(θ_new,λ,ζ)
    return adjust_angular_symm_val!(g_value,θ_val_new,θ_val_old,exp_new,exp_old,tpz)
end

function adjust_radial_symm_val!(g_value1,g_value2,rnew_ij,r2_ij,fnew_ij,f2_ij,η)
 
    g_value1,g_value2 = adjust_symm_val!(g_value1,rnew_ij,fnew_ij,η),adjust_symm_val!(g_value2,rnew_ij,fnew_ij,η)
    g_value1,g_value2 = adjust_symm_val!(g_value1,r2_ij,-f2_ij,η),adjust_symm_val!(g_value2,r2_ij,-f2_ij,η)

    return g_value1,g_value2
end

adjust_radial_symm_val! (generic function with 1 method)

Using these adjust functions we define the calc_symm functions

In [10]:
#most general definition for the angular value

function calc_new_symmetry_value!(g_vector,indices,newposition,position1,position2,position3,rnew_ij,rnew_ik,r2_ij,r2_ik,r2_jk,fnew_ij,fnew_ik,f_ij,f_ik,f_jk,η,λ,ζ,tpz)

    θ_new_vec,θ_old_vec = all_angular_measure(newposition,position2,position3,rnew_ij,rnew_ik,r2_jk),all_angular_measure(position1,position2,position3,r2_ij,r2_ik,r2_jk)

    exp_new,exp_old = exponential_part(η,rnew_ij,rnew_ik,r2_jk,fnew_ij,fnew_ik,f_jk),exponential_part(η,r2_ij,r2_ik,r2_jk,f_ij,f_ik,f_jk)

    for (θ_old,θ_new,index) in zip(θ_old_vec,θ_new_vec,indices)
        g_vector[index] = adjust_angular_symm_val!(g_vector[index],exp_old,exp_new,θ_old,θ_new,λ,ζ,tpz)
    end

    return g_vector
end
function calc_new_symmetry_value!(g_vector,indexi,indexj,indexk,newposition,position,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,η,λ,ζ,tpz)
    return calc_new_symmetry_value!(g_vector,[indexi,indexj,indexk],newposition,position[indexi],position[indexj],position[indexk],new_dist2_vector[indexj],new_dist2_vector[indexk],dist2_mat[indexi,indexj],dist2_mat[indexi,indexk],dist2_mat[indexj,indexk],new_f_vector[indexj],new_f_vector[indexk],f_matrix[indexi,indexj],f_matrix[indexi,indexk],f_matrix[indexj,indexk],η,λ,ζ,tpz)

end

function calc_new_symmetry_value!(g_vector,indexi,indexj,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,η)
    g_vector[indexi],g_vector[indexj] = adjust_radial_symm_val!(g_vector[indexi],g_vector[indexj],new_dist2_vector[indexj],dist2_mat[indexi,indexj],new_f_vector[indexj],f_matrix[indexi,indexj],η)
    return g_vector
end

calc_new_symmetry_value! (generic function with 3 methods)

Now a function to decide between radial and angular versions of the calc_new_symm_value! function. 

In [11]:
function symmetry_calculation!(g_vector,atomindex,newposition,position,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,symmetry_function::RadialType2)
    if symmetry_function.type_vec == [1.,1.]

        η = symmetry_function.eta
        for index2 in eachindex(g_vector)
            if index2 != atomindex
                g_vector = calc_new_symmetry_value!(g_vector,atomindex,index2,dist2_mat,new_dist2_vector,f_matrix,new_f_vector,η)
            end
        end
    end

    return g_vector
end
function symmetry_calculation!(g_vector,atomindex,newposition,position,dist2_mat,new_dis_vector,f_matrix,new_f_vector,symmetry_function::AngularType3)
    N = length(g_vector)

    if symmetry_function.type_vec == [1.,1.,1.]

        η,λ,ζ,tpz = symmetry_function.eta,symmetry_function.lambda,symmetry_function.zeta,symmetry_function.tpz

        for j_index in 1:N
            if j_index != atomindex
                for k_index in j_index+1:N
                    if k_index != atomindex
                        g_vector = calc_new_symmetry_value!(g_vector,atomindex,j_index,k_index,newposition,position,dist2_mat,new_dis_vector,f_matrix,new_f_vector,η,λ,ζ,tpz) 
                    end
                end
            end
        end
        
    end

    return g_vector
end

symmetry_calculation! (generic function with 2 methods)

As a means of testing the accuracy, we will generate a single difference vector and check whether this matches

In [12]:
vector_for_testing = zeros(55)

vector_for_testing = symmetry_calculation!(vector_for_testing,3,newpos,positions,dis2mat,new_dis_vec,f_mat,new_f_vec,angularsymmvec[1])
gmat2[11,:] .- gmat1[11,:] ≈ vector_for_testing

true

In [13]:
@benchmark symmetry_calculation!($zeros(55),$3,$newpos,$positions,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$angularsymmvec[1])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   99.683 μs …   5.357 ms  ┊ GC (min … max):  0.00% … 96.83%
 Time  (median):     111.525 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   147.072 μs ± 298.629 μs  ┊ GC (mean ± σ):  12.34% ±  5.93%

  ▁▆▇█▅▂                                                         
  ██████▇▅▅▅▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▄▅▆▄▄▃▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁ ▂
  99.7 μs          Histogram: frequency by time          212 μs <

 Memory estimate: 335.98 KiB, allocs estimate: 4296.

Lastly, a function that handles the complete change to the symmetry matrix based on each and every vector. 

In [14]:
function total_symm!(g_matrix,position,new_position,dist2_matrix,new_dist_vector,f_matrix,new_f_vector,atomindex,total_symmetry_vector)
    for g_index in eachindex(total_symmetry_vector)
        g_matrix[g_index,:] = symmetry_calculation!(g_matrix[g_index,:],atomindex,new_position,position,dist2_matrix,new_dist_vector,f_matrix,new_f_vector,total_symmetry_vector[g_index])
    end

    return g_matrix
end

total_symm! (generic function with 1 method)

The _modified_ g_matrix should match the one we calculated above to represent the values at the new positions. 

In [15]:
test_g_matrix = copy(gmat1)
test_g_matrix = total_symm!(test_g_matrix,positions,newpos,dis2mat,new_dis_vec,f_mat,new_f_vec,3,total_symm_vec)
test_g_matrix ≈ gmat2

true

This final benchmark shows a massive improvement to the calculation of the symmetry function. Down to 3.6ms 

In [16]:
@benchmark total_symm!($zeros(88,55),$positions,$newpos,$dis2mat,$new_dis_vec,$f_mat,$new_f_vec,$3,$total_symm_vec)

BenchmarkTools.Trial: 1246 samples with 1 evaluation.
 Range (min … max):  2.819 ms … 10.091 ms  ┊ GC (min … max):  0.00% … 48.50%
 Time  (median):     3.176 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.011 ms ±  1.758 ms  ┊ GC (mean ± σ):  12.04% ± 16.39%

  ▃██▆▅▃▁         ▂▄▄▁                                        
  ████████▇█▇▇██▇█████▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▅▆█▇▇▇▇█▇▇▆█▇▅█ █
  2.82 ms      Histogram: log(frequency) by time     9.61 ms <

 Memory estimate: 8.60 MiB, allocs estimate: 111884.